# <font color="green">YOLO V2
* 原文地址：
    + [You Only Look Once: Unified, Real-Time Object Detection](https://arxiv.org/abs/1506.02640)
    + [YOLO9000: Better, Faster, Stronger](https://arxiv.org/abs/1612.08242)
* 笔记精选
    + [论文阅读：You Only Look Once: Unified, Real-Time Object Detection](http://blog.csdn.net/u010167269/article/details/52638771)
    + [YOLO V2论文笔记](http://blog.csdn.net/jesse_mx/article/details/53925356)


## <font color="green">YOLO
    
![](http://zh.gluon.ai/_images/yolo.svg)
    
**YOLO的核心思想**：从R-CNN到Fast R-CNN一直采用的是proposal+分类（proposal提供位置信息，分类提供类别信息），精度已经很高，但是速度还是不行。YOLO提供了另一种更为直接的思路：直接在输出层回归bounding box的位置和boundin box所属的类别，即把整张图作为网络的输入，把object detection的问题转化成一个regression的问题。

**YOLO的主要特点**：
* 速度快，能够达到实时的要求，在Titan X的GPU上能够达到45帧每秒。
* 使用全图作为Context信息，背景错误（把背景错认为物体）比较少。
* 泛化能力比较强。

**YOLO的主要流程：**

* 1.Resize成$448 \times 448$, 图片分得到$7 \times 7$的网格，
* 2.CNN提取特征和预测：卷积部分负责提取特征，全连接部分负责预测，
    + $7 \times 7 \times 2 = 98$个bounding box的坐标$[x\_center, y\_center, w, h]$和是否有物体的confidence。
    + $7 \times 7 = 49$个cell所属的20个物体的概率。
* 3.通过NMS过滤bounding box。

**网络设计**

<img src="../img/Chapter8-Computer-Vision/8-3.png" width="700">

网络结构借鉴了GoogleNet,24个卷积层，2个全连接层。(用$1 \times 1$ reduction layers紧跟 $3 \times 3$ convolution layers取代GoogleNet的inception modules)。

**训练** 

* 预训练分类网络：在ImageNet上预训练一个分类网络，这个网络是上图中的前20个卷积网络+avg pooling layer+fully connected layer（此时网络的输入是$224\times224$)。

* 训练检测网络：转换模型去执行检测任务，通常在预训练网络中增加卷积和全连接层可以改善性能，因此在预训练的分类网络的基础上添加4个卷积层和2个全连接层，随机初始化权重。**又因为检测要求细粒度的视觉信息，所以把网络输入从$224 \times 224$变成$448 \times 448$。**
    + 一幅图片分成$7 \times 7$个网格，某个物体的中心落在这个网格中此网格就负责预测这个物体。
    + 最后一层输出为$（7 \times 7）\times 30$的维度，每个$1 \times 1 \times 30$的维度对应原图$7 \times 7$cell中的一个，$1 \times 1 \times 30$中含有类别预测和bbox坐标预测，总的来讲就是让网格负责类别信息。
    
 <img src="../img/Chapter8-Computer-Vision/8-4.jpg" width="300">
    
    + 每个网格($1 \times 1 \times 30$维度对应原图中的cell)要预测2个bounding box（图中黄色实线框）的坐标($x\_center, y\_center, w, h$)，其中，中心坐标的$x\_center, y\_center$相对于对应的网格归一化到0-1之间，w, h用图像的width和height归一化到0-1之间。每个bounding box除了要回归自身的位置之外，还要附带预测一个confidence值。这个confidence代表了所预测的box中含有object的置信度和这个box预测的有多准两重信息，即：
    
    $$confidence = Pr(Object) * IOU^{truth}_{pred}$$
    
    其中如果有ground truth box(人工标记的物体)落在一个grid cell里，第一项取1，否则取0。第二项是预测的bounding box和实际的ground truth box之间的IOU值。即：每个bounding box和实际要预测$x\_center, y\_center, w, h, confidence$，共5个值，2个bounding box共10个值，对应$1 \times 1 \times 30$维度特征中的前10个。
       
    + 每个网格还要预测类别细信息，论文中有20个类，$7 \times 7$的网格，每个网格要预测2个bounding box和20个类别概率，输出就是$7 \times 7 \times (5 \times 2 + 20)$，**<font color="red">注意class信息是针对每个网络的，confidence信息是针对每个bounding box的**。
 
**损失函数的设计**

<img src="../img/Chapter8-Computer-Vision/8-5.jpg" width="700">

损失函数的设计就是让坐标(x, y, w, h), confidence, classification这三个方面达到很好的平衡。简单的全部采用sum-square error loss来做这件事会有以下不足：
* 8维的localization error和20的classification error同等重要显然是不合理；
* 如果一个网格中没有object(一幅图中这种网格很多)那么就会将这些网格中box和confidence push为0，相比于较少的有object的网格，这种做法显然太过，这会导致网络不稳定甚至发散，解决方案如下：
 
    * 更重视8维的坐标预测，给这些损失前面赋予更大的loss weight, 记为 $\lambda_{coord}$ ,在pascal VOC训练中取5。（上图蓝色框）
    * 对没有object的bbox的confidence loss，赋予小的loss weight，记为 $\lambda_{noobj}$ ，在pascal VOC训练中取0.5。
    * 有object的bbox的confidence loss (上图红色框) 和类别的loss （上图紫色框）的loss weight正常取1。
    * 对不同大小的bbox预测中，相比于大bbox预测偏一点，小box预测偏一点更不能忍受。而sum-square error loss中对同样的偏移loss是一样。 为了缓和这个问题，作者用了一个比较取巧的办法，就是将box的width和height取平方根代替原本的height和width。 如下图：small bbox的横轴值较小，发生偏移时，反应到y轴上的loss（下图绿色）比big box(下图红色)要大。
    * 一个网格预测多个bounding box，在训练时我们希望每个object（ground truth box）只有一个bounding box专门负责（一个object 一个bbox）。具体做法是与ground truth box（object）的IOU最大的bounding box 负责该ground true box(object)的预测。这种做法称作bounding box predictor的specialization(专职化)。每个预测器会对特定（sizes, aspect ratio or classed of object）的ground truth box预测的越来越好。（个人理解：IOU最大者偏移会更少一些，可以更快速的学习到正确位置）
    
<img src="../img/Chapter8-Computer-Vision/8-6.jpg" width="600">

**测试**

测试的时候，每个网格预测的class信息( $Pr(Class_i | Object)$ )和bounding box预测的confidence信息( $Pr(Object) \ast IOU^{truth}_{pred}$ ) 相乘，就得到每个bounding box的class-specific confidence score。

$$Pr(Class_i | Object) \ast Pr(Object) \ast IOU^{truth}_{pred} = Pr(Class_i) \ast IOU^{truth}_{pred}$$

* 等式左边第一项就是每个网格预测的类别信息，第二三项就是每个bounding box预测的confidence。这个乘积即encode了预测的box属于某一类的概率，也有该box准确度的信息。
* 对每一个网格的每一个bbox执行同样操作：7x7x2 = 98 bbox（每个bbox既有对应的class信息又有坐标信息）
* 得到每个bbox的class-specific confidence score以后，设置阈值，滤掉得分低的boxes，对保留的boxes进行NMS处理，就得到最终的检测结果。

**缺陷**

* YOLO对相互靠的很近的物体（挨在一起且中点都落在同一个格子上的情况），还有很小的群体 检测效果不好，这是因为一个网格中只预测了两个框，并且只属于一类。
* 测试图像中，当同一类物体出现的不常见的长宽比和其他情况时泛化能力偏弱。
* 由于损失函数的问题，定位误差是影响检测效果的主要原因，尤其是大小物体的处理上，还有待加强。

## <font color="green">YOLO9000 : Better, Faster, Stronger
    
**<font color="red">1.概述**

YOLO V2主要有两个大方面的改进：
* 使用一系列的方法对YOLO进行了改进，在保持原有速度的同时提升精度得到YOLO V2。
* 提出了一种目标分类与检测的联合训练方法，同时在COCO和ImageNet数据集中进行训练得到YOLO 9000，实现9000多种物体的实时检测。

**<font color="red">2.更精确**

**<font color="red">Batch Normalization**

神经网络学习过程本质就是为了学习数据分布，一旦训练数据与测试数据的分布不同，那么网络的泛化能力也大大降低；另一方面，一旦每批训练数据的分布各不相同，那么网络就要在每次迭代都去适应不同的分布，这样会大大降低网路的训练速度。

解决办法之一是对数据都要做归一化预处理。YOLO V2网络通过在每一个卷积层后添加batch normalization，极大的改善了收敛速度同时减少了对其他regularization方法的依赖（舍弃了dropout优化后依然没有过拟合），使得mAP获得了2%的提升

**<font color="red"><font color="red">High Resolution CLassifier**

所有state-of-the-art的检测方法基本上都会使用ImageNet预训练过的模型来提取特征，例如AlexNet输入图片会被resize到不足$224 \times 224$，这导致分辨率不够高，给检测带来困难。所以YOLO(v1)先以分辨率$224 \times 224$训练分类网络，然后需要增加分辨率到$448 \times 448$，这样做不仅切换为检测算法也改变了分辨率。所以作者想能不能在预训练的时候就把分辨率提高了，训练的时候只是由分类算法切换为检测算法。

YOLO V2首先修改预训练分类网络的分辨率为$448 \times 448$，在ImageNet数据集上训练10轮（10 epochs）。这个过程让网络有足够的时间调整filter去适应高分辨率的输入。然后fine tune为检测网络。mAP获得了4%的提升。

**<font color="red">Convolutional with anchor boxes**

YOLO(v1)使用全连接层数据进行bounding box预测（要把1470*1的全链接层reshape为7*7*30的最终特征），这会丢失较多的空间信息定位不准。YOLOv2借鉴了Faster R-CNN中的anchor思想： 简单理解为卷积特征图上进行滑窗采样，每个中心预测9种不同大小和比例的建议框。由于都是卷积不需要reshape，很好的保留的空间信息，最终特征图的每个特征点和原图的每个cell一一对应。**而且用预测相对偏移（offset）取代直接预测坐标简化了问题，方便网络学习**。

总的来说就是移除全连接层（以获得更多空间信息）使用 anchor boxes 取预测 bounding boxes。具体做法如下：
* 去掉最后的池化层确保输出的卷积特征图有更高的分辨率。
* 缩减网络，让图片输入分辨率为$416 \times 416$，目的是让后面产生的卷积特征图宽高都为奇数，这样就可以产生一个center cell。因为作者观察到，大物体通常占据了图像的中间位置，可以只用一个中心的cell来预测这些物体的位置，否则就要用中间的4个cell来进行预测，这个技巧可稍稍提升效率。
* 使用卷积层降采样（factor为32），使得输入卷积网络的$416 \times 416$图片最终得到13 * 13的卷积特征图（416/32=13）。
* 把预测类别的机制从空间位置(cell)中解耦，由anchor box同时预测类别和坐标。因为YOLO是由每个cell来负责预测类别，每个cell对应的2个bounding box负责预测坐标（回想YOLO中最后输出7*7*30的特征，每个cell对应1*1*30，前10个主要是2个bounding box用来预测坐标，后20个表示该cell在假设包含物体的条件下属于20个类别的概率。YOLOv2中，不再让类别的预测与每个cell（空间位置）绑定一起，而是让全部放到anchor box中。下面是特征维度示意图（仅作示意并非完全正确）

<img src="../img/Chapter8-Computer-Vision/8-7.jpg" width="600">

加入了anchor boxes后，可以预料到的结果是召回率上升，准确率下降。我们来计算一下，假设每个cell预测9个建议框，那么总共会预测$13 * 13 * 9 = 1521$个boxes，而之前的网络仅仅预测$7 * 7 * 2 = 98$个boxes。具体数据为：没有anchor boxes，模型recall为81%，mAP为69.5%；加入anchor boxes，模型recall为88%，mAP为69.2%。这样看来，准确率只有小幅度的下降，而召回率则提升了7%，说明可以通过进一步的工作来加强准确率，的确有改进空间。

**<font color="red">Dimension Clusters**

使用anchor时，作者发现Faster-RCNN中anchor boxes的个数和宽高维度往往是手动精选的先验框（hand-picked priors)，设想能否一开始就选择了更好的、更有代表性的先验boxes维度，那么网络就应该更容易学到准确的预测位置。解决办法就是统计学习中的K-means聚类方法，通过对数据集中的ground truth box做聚类，找到ground truth box的统计规律。**<font color="green">以聚类个数k为anchor boxs个数，以k个聚类中心box的宽高维度为anchor box的维度**。
    
如果按照标准k-means使用欧式距离函数，大boxes比小boxes产生更多error。但是，我们真正想要的是产生好的IOU得分的boxes（与box的大小无关）。因此采用了如下距离度量：

$$d(box, centroid) = 1 - IOU(box, centroid)$$

最后通过维度聚类选取的锚框数为5。

**<font color="red">Direct Location Prediction**

在使用anchor box时的另一个问题是模型不够稳定，尤其在早期迭代的时候，大部分的不稳定现象出现在预测box的（x,y）坐标时。在区域建议网络（RPN）中会预测坐标就是预测tx，ty。对应的中心点（x,y）按如下公式计算：

$$x = (t_x \ast w_a) + x_a, y = (t_y \ast h_a) + y_a$$

可见预测tx=1就会把box向右移动anchor box的宽度，预测tx=-1就会把box向左移动相同的距离。

这个公式没有任何限制，无论在什么位置进行预测，任何anchor boxes可以在图像中任意一点。模型随机初始化之后将需要很长一段时间才能稳定预测敏感的物体偏移。因此作者没有采用这种方法，而是预测相对于grid cell的坐标位置，同时把ground truth限制在0到1之间（利用logistic激活函数约束网络的预测值来达到此限制）。

最终，网络在特征图$13 \times 13$的每个cell上预测5个bounding boxes，每一个bounding box预测5个坐标值：$t_x，t_y，t_w，t_h，t_o$。如果这个cell距离图像左上角的边距为$（cx，cy）$以及该cell对应的box维度（bounding box prior，指通过维度聚类选取的锚框）的长和宽分别为（$p_w，p_h$），那么对应的box为：

$$b_x = \sigma(t_x) + c_x$$
$$b_y = \sigma(t_y) + c_y$$
$$b_w = p_w e^{t_w}$$
$$b_h = p_h e^(t_h)$$
$$Pr(object) \ast IOU(b, object) = \sigma(t_o)$$

<img src="../img/Chapter8-Computer-Vision/8-8.png" width="400">

约束了位置预测的范围，参数就更容易学习，模型就更稳定。使用Dimension Clusters和Direct location prediction这两项anchor boxes改进方法，mAP获得了5%的提升。

**<font color="red">Fine-Grained Features**

修改后的网络最终在$13 \times 13$的特征图上进行预测，虽然这足以胜任大尺度物体的检测，如果用上细粒度特征的话可能对小尺度的物体检测有帮助。Faser R-CNN和SSD都在不同层次的特征图上产生区域建议以获得多尺度的适应性。YOLO V2使用了一种不同的方法，简单添加一个passthrough layer，把浅层特征图（分辨率为$26 \times 26$）连接到深层特征图。

passthroughlaye把高低分辨率的特征图做连结，叠加相邻特征到不同通道（而非空间位置），类似于Resnet中的identity mappings。这个方法把$26 \times 26 \times 512$的特征图叠加成$13 \times 13 \times 2048$的特征图，与原生的深层特征图相连接。

YOLOv2的检测器使用的就是经过扩展后的的特征图，它可以使用细粒度特征，使得模型的性能获得了1%的提升。

**<font color="red">Multi-Scale Training**

原始YOLO网络使用固定的$448 \times 448$的图片作为输入，加入anchor boxes后输入变成$416 \times 416$，由于网络只用到了卷积层和池化层，就可以进行动态调整（检测任意大小图片）。为了让YOLOv2对不同尺寸的图片具有鲁棒性，在训练的时候也考虑了这一点。

不同于固定网络输入图片尺寸的方法，每经过10批训练（10 batches）就会随机选择新的图片尺寸。网络使用的降采样参数为32，于是使用32的倍数{320,352，…，608}，最小的尺寸为$320 \times 320$，最大的尺寸为$608 \times 608$。 调整网络到相应维度然后继续进行训练。

这种机制使得网络可以更好地预测不同尺寸的图片，同一个网络可以进行不同分辨率的任务，在小尺寸图片上YOLOv2运行更快，在速度和精度上达到了平衡。

在低分辨率图片检测中，YOLOv2是检测速度快（计算消耗低），精度较高的检测器。输入为$228 \times 228$的时候，帧率达到90FPS，mAP几乎和Faster R-CNN的水准相同。使得其更加适用于低性能GPU、高帧率视频和多路视频场景。

在高分辨率图片检测中，YOLOv2达到了先进水平（state-of-the-art），VOC2007上mAP为78.6%，而且超过实时速度要求。

-------------------------------------------------------------------------------------------------------------------------------

**<font color="red">自己总结**

YOLO V2对YOLO进行一些地方的改进，其主要包括：
* 1，使用更好的卷积神经网络来做特征提取，使用更大的输入图片$448 \times 448$使得特征输出大小增加到$13 \times 13$
* 2.不再使用均匀切分的锚框，而是对训练数据里的真实锚框做聚类，然后使用聚类中心作为锚框，相对于SSD和Faster R-CNN可以说是大幅降低锚框的个数。
* 3.不再使用YOLO的全连接层来预测，而是同SSD一样使用卷积。例如假设使用5个锚框（聚类为5类），那么物体分类使用通道数是$5 \times (num\_classes + 1)$的$1 \times 1$卷积，边框回归使用通道数$4 \times 5$。

具体来说，YOLO V2改进的几个点：
* 1.使用BN的技巧对设计的网络施加正则，使得网络更快和更好的收敛。
* 2.使用更高分辨率的图像作为网络的输入，网络可以从高分辨率图像中获得更大的性能提升。
* 3.移除全连接层，使用anchor boxes来获得边框准确率的提升
* 4.摒弃类似于Faster R-CNN和SSD中手工选取的先验锚框，采用维度聚类， 利用以IOU为评判标准的改进的K-Means聚类训练bounding box，从而获得更好的锚框。
* 5.采用直接位置预测，利用sigmod将ground truth限制在0-1之间，使得每个cell所产生的锚框只负责其周围一个单位以内的目标box。
* 6.提出passthrough layer转移层，利用ResNet中identity mapping的思想来讲浅层特征图连接到深层特征图，完成多尺度预测，有利于检测小目标。
* 7.针对不同输入尺寸图片，微调网络，使得网络可以更好的预测不同尺度的图片。

--------------------------------------------------------------------------------------------------------------------------------

In [2]:
import mxnet as mx
import numpy as np

from mxnet import nd
from mxnet import gluon
from mxnet import image
from mxnet import metric
from mxnet import autograd
from mxnet import lr_scheduler
from mxnet.gluon.model_zoo import vision as models


%matplotlib inline 
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 120
import matplotlib.pyplot as plt

/home/liang/.virtualenvs/liang-jupyter-2/lib/python3.5/site-packages/mxnet/optimizer.py:136: UserWarning: WARNING: New optimizer mxnet.optimizer.NAG is overriding existing optimizer mxnet.optimizer.NAG
  Optimizer.opt_registry[name].__name__))


## <font color="green">原始卷积输出的转换

我们首先定义原始特征提取网络

In [ ]:
# pretrained features 
pretrained = models.get_model('resnet18_v1', pretrained=True).features

finetune_net = gluon.nn.HybridSequential()
for i in range(len(pretrained) - 2):
    finetune_net.add(pretrained[i])
    
# Now, if we use the input X : (1, 3, 256, 256), then the output will be (1, 256, 16, 16)

anchor_scales = [[3.3004, 3.59034], [9.84923, 8.23783]]

predictor = Yolov2Output(num_classes=2, anchor_scales=anchor_scales)
predictor.initialize()
finetune_net.add(predictor)

# Now, if we use the input X : (1, 3, 256, 256), then the output will be (1, 14, 16, 16)

## <font color="green">YOLO V2的输出

我们定义YOLO V2的输出函数，它将原始图片经特征提取层(e.g. darknet, resnet, densenet pretrained on imagenet)的输出结果作为输入，并通过一个$1 \times 1$卷积转换输出，输出相关预测量

In [90]:
class Yolov2Output(gluon.nn.HybridBlock):
    def __init__(self, num_classes, anchor_scales, **kwargs):
        '''
        num_classes : 给定的类别数
        anchor_scales : 经k-means输出选定的锚框的尺寸
        '''
        super().__init__(**kwargs)
        # assertion
        assert num_classes > 0, 'Number of classes should over zero, but given {}.'.format(num_classes)
        assert len(anchor_scales) > 0, 'At least one anchor scale required.'
        for anchor in anchor_scales:
            assert len(anchor) == 2, 'Expected each anchor scale to be (width, height), but given {}.'.format(anchor)
        
        self._num_classes = num_classes
        self._anchor_scales = anchor_scales
        num_anchors_per_pixel = len(anchor_scales)
        
        # 每个cell会产生num_anchors_per_pixel个锚框，而每个锚框需要预测num_classes + 1 + 4个类别
        # 其中num_classes表示类别数，1表示背景，4表示coordinates
        out_channels = num_anchors_per_pixel * (num_classes + 1 + 4)
        with self.name_scope():
            self.output = gluon.nn.Conv2D(out_channels, kernel_size=1, strides=1)
            
    def hybrid_forward(self, F, X):
        return self.output(X)

## <font color="green">定义前向函数

``slice_axis``用法： 沿着给定的axis切分某个NDArray
    * data : 指定要切割NDArray
    * axis : 指定要切割的axis
    * begin :  从该axis开始切割的index（从0开始）
    * end : 从该axis结束切割的index（不包括该index）

### 中心点与长宽的转换

YOLO V2同样没有预测偏移，而是遵循YOLO的方法预测相对于grid cell的位置坐标，因此我们需要对网络的输出进行转换。

<img src="http://img.blog.csdn.net/20161229113953466?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvSmVzc2VfTXg=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70/gravity/SouthEast" width="500">

中心点是sigmoid函数的输出值，本身在0到1之间，我们需要将其转换为相对于整个feature_map的坐标

In [27]:
example1 = nd.arange(0, 16, repeat=16*2*1).reshape((1, 16, 16, 2, 1))
# print(example1)
print(example1.shape)

(1, 16, 16, 2, 1)


In [28]:
example1 = nd.tile(example1, reps=(2,1,1,1,1))
# print(example1)
print(example1.shape)

(2, 16, 16, 2, 1)


In [29]:
# 中心点的转换 将x, y的坐标转换为相对于当前feature map左上角的实际值
# direct location prediction
def transform_center(xy): # (1, 16, 16, 2, 2)
    # xy此时是我们sigmoid至[0,1]之间的值，也是每个anchor box的中心点的坐标
    # batch_sie, height, width, num_anchors_per_pixel, size
    b, h, w, n, _ = xy.shape 
    
    # compute offset
    continuous_y = nd.arange(0, h, repeat=(w*n*1), ctx=xy.context).reshape((1, h, w, n, 1))
    offset_y = nd.tile(continuous_y, (b, 1, 1, 1, 1))
    continuous_x = nd.arange(0, w, repeat=(n*1), ctx=xy.context).reshape((1, 1, w, n, 1))
    offset_x = nd.tile(continuous_x, (b, h, 1, 1, 1))
    
    # Splits an array along a particular axis into multiple sub-arrays.
    x, y = xy.split(num_outputs=2, axis=-1)
    # 除以图片格点的数量
    x = (x + offset_x) / w
    y = (y + offset_y) / h
    return x, y

In [30]:
example_anchor_scales = [[3.3004, 3.59034], [9.84923, 8.23783]]
example_anchor_scales = nd.array(example_anchor_scales).reshape((1, 1, 1, -1, 2))
example_aw, example_ah = nd.tile(example_anchor_scales, reps=(2, 16, 16, 1, 1)).split(num_outputs=2, axis=-1)
print(example_aw.shape)
print(example_ah.shape)
# print(example_ah)
# print(example_aw)

(2, 16, 16, 2, 1)
(2, 16, 16, 2, 1)


In [31]:
# 长宽的转换 将预测的长宽转换为实际的比例，并乘以原来的长宽，得到新的anchor box的长宽
def transform_size(wh, anchor_scales):
    b, h, w, n, _ = wh.shape
    anchor_scales = nd.array(anchor_scales).reshape((1, 1, 1, -1, 2))
    aw, ah = nd.tile(anchor_scales, reps=(b, h, w, 1, 1)).split(num_outputs=2, axis=-1)
    w_pred, h_pred = nd.exp(wh).split(num_outputs=2, axis=-1)
    w_out = w_pred * aw / w
    h_out = h_pred * ah / h
    return w_out, h_out

### 前向函数

In [16]:
x = nd.random.normal(shape=(1, 16, 16, 2, 7))
x1 = x.slice_axis(begin=0, end=2, axis=-1)
x1.shape

(1, 16, 16, 2, 2)

In [34]:
def yolov2_forward(input_X, num_classes, anchor_scales):
    # suppose the input X shape is 1 X 2(2+1+4) X 16 X 16, num_classes=2, len(anchor_scales)=2
    channels = num_classes + 5
    # 将通道层转置到最后一个axis
    input_X = input_X.transpose((0, 2, 3, 1)) # batch X height X width X channel (1, 16, 16, 14)
    # the 4th dim is the number of anchors
    input_X = input_X.reshape((0, 0, 0, -1, channels)) # (1, 16, 16, 2, 7)
    
    # 下面我们要做的就是切割X，因为该X的最后一个dim包含了num_classes + objectness_score + coordinates
    # class_preds
    cls_preds = input_X.slice_axis(axis=-1, begin=0, end=num_classes) # (1, 16, 16, 2, 2)
    
    # obejctness_score
    score_preds = input_X.slice_axis(axis=-1, begin=num_classes, end=num_classes+1) # (1, 16, 16, 2, 1)
    score = nd.sigmoid(score_preds)
    
    # x, y coordinates
    xy_preds = input_X.slice_axis(axis=-1, begin=num_classes+1, end=num_classes+3) # (1, 16, 16, 2, 2)
    # 将预测得到的每个anchor box的中心点坐标限定在0-1之间以保证每个anchor box只预测周围一个单位以内
    xy = nd.sigmoid(xy_preds)
    
    # 预测以当前cell为中心的所有anchor box的长宽 
    wh = input_X.slice_axis(axis=-1, begin=num_classes+3, end=input_X.shape[-1]) # (1, 16, 16, 2, 2)
    
    # 将[0,1]之间的x,y坐标转换成相对于当前feature_map的真实坐标
    x, y = transform_center(xy)
    
    # 将预测的w, h根据anchor_scales转换为真实的w, h
    w, h = transform_size(wh, anchor_scales)
    
    # 利用argmax预测所有anchor box所代表的类别
    cid = nd.argmax(cls_preds, axis=-1, keepdims=True)
    
    # convert to corner format boxes 将中心坐标转换为左上坐标、右下坐标的格式
    # 此时x,y是当前anchor box的中心坐标， w,h是当前anchor box的宽高
    half_w = w / 2
    half_h = h / 2
    
    # 得到左上角的坐标
    left = nd.clip(x - half_w, 0, 1)
    top = nd.clip(y - half_h, 0, 1)
    
    # 得到右下角的坐标
    right = nd.clip(x + half_w, 0, 1)
    bottom = nd.clip(y + half_h, 0, 1)

    # concat
    output = nd.concat(*[cid, score, left, top, right, bottom], dim=4)
    return output, cls_preds, nd.concat(*[xy, wh], dim=4)

In [38]:
X = nd.random.normal(shape=(1, 14, 16, 16))
a, b, c = yolov2_forward(X, num_classes=2, anchor_scales=example_anchor_scales)
a.shape

(1, 16, 16, 2, 6)

## <font color="green">生成真实目标
    
YOLO2寻找真实目标的方法比较特殊，是在每个格点内各自比较，而不是使用全局的预设。而且我们不需要对生成的训练目标进行反向传播，为了简洁描述比较的方法，我们可以在这里转成numpy而且可以用for循环（切记转成numpy会破坏自动求导的记录，只有当反向传播不需要的时候才能使用这个技巧），实际使用中，如果遇到速度问题，我们可以用mx.ndarray矩阵的写法来加速。 这里我们使用了一个技巧：sample_weight（个体权重）矩阵， 用于损失函数内部权重的调整，我们也可以通过权重矩阵来控制哪些个体需要被屏蔽，这一点在目标检测中尤其重要，因为往往大多数的背景区域不需要预测检测框。

In [ ]:
def yolov2_target(scores, boxes, labels, anchors, ignore_labels=-1, thresh=0.5):
    b, h, w, n, _ = scores.shape
    anchors = np.reshape(np.array(anchors), (-1, 2))
    
    # ground truth box
    gt_boxes = nd.slice_axis(labels, begin=1, end=5, axis==-1)
    
    target_score = nd.zeros((b, h, w, n, 1), ctx=scores.context)
    target_id = nd.ones_like(target_score, ctx=scores.context) *ignore_labells
    target_box = nd.zeros((b, h, w, n, 4), ctx=scores.context)
    
    # 个体权重矩阵
    sample_weight = nd.zeros((b, h, w, n, 1), ctx=scores.context)
    
    # output :  output = nd.concat(*[cid, score, left, top, right, bottom], dim=4)
    # if we use the input_X = (1, 14, 16, 16), then the output shape is (1, 16, 16, 2, 6)
    for b in range(output.shape[0]):
        label = labels[b].asnumpy)()
        valid_label = label[np.where(label[:, 0] > -0.5)[0], :]
        pass

In [26]:
# pretrained features 
pretrained = models.get_model('resnet18_v1', pretrained=True).features

finetune_net = gluon.nn.HybridSequential()
for i in range(len(pretrained) - 2):
    finetune_net.add(pretrained[i])
    
# Now, if we use the input X : (1, 3, 256, 256), then the output will be (1, 256, 16, 16)

anchor_scales = [[3.3004, 3.59034], [9.84923, 8.23783]]

predictor = Yolov2Output(num_classes=2, anchor_scales=anchor_scales)
predictor.initialize()
finetune_net.add(predictor)

# Now, if we use the input X : (1, 3, 256, 256), then the output will be (1, 14, 16, 16)

## <font color="green">训练

In [ ]:
def train_yolo():
    pass

## <font color="green"> 测试